In [1]:
from pyproj import crs
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import matplotlib.pyplot as plt
import folium
import os

In [2]:
# path UKR - Gazetteer of locations for FAD - 19052022.XLSX
fp = r'UKR_data_sources/UKR_gazeteer/UKR - Gazetteer of locations for FAD - 19052022.XLSX'

# read HDX admin 4 polygon shapefile into a geodataframe
df_fad_gaz = pd.read_excel(fp, dtype=object)

In [3]:
# UKR - Locations pick list values for FAD - 27052022.XLSX
fp = r'UKR_data_sources/UKR_gazeteer/UKR - Locations pick list values for FAD - 27052022.XLSX'

# read HDX admin 4 polygon shapefile into a geodataframe
df_fad_picklist = pd.read_excel(fp, dtype=object)

In [4]:
# DEFINE GEOMETRY
# create a geodataframe from the iBase gazetteer

# Define a function for creating points from row values
def create_point(row):
    # Returns a shapely point object based on values in x and y columns

    point = Point(row['lon'], row['lat'])

    return point

# Apply the function to each row
df_fad_gaz['geometry'] = df_fad_gaz.apply(create_point, axis=1)
df_fad_picklist['geometry'] = df_fad_picklist.apply(create_point, axis=1)

C:\Users\Billen\Anaconda3\envs\py38_gis\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\Billen\Anaconda3\envs\py38_gis\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [5]:
# IBASE_GAZ DATAFRAME
# Convert IBASE_GAZ DataFrame into a GeoDataFrame
geo_fad_gaz = gpd.GeoDataFrame(df_fad_gaz, geometry='geometry', crs="EPSG:4326")
geo_fad_picklist = gpd.GeoDataFrame(df_fad_picklist, geometry='geometry', crs="EPSG:4326")

In [13]:
geo_fad_picklist.columns

Index(['Name', '[Country (Admin Level 1)]', '[Country (Admin Level 2)]',
       '[Country (Admin Level 3)]', '[Country (Admin Level 4)]',
       '[Country (Admin Level 5)]', 'lat', 'lon', 'geometry'],
      dtype='object')

In [20]:
columns = ['Name', '[Country (Admin Level 1)]', '[Country (Admin Level 2)]',
           '[Country (Admin Level 3)]', '[Country (Admin Level 4)]',
           '[Country (Admin Level 5)]', 'geometry']

In [21]:
geo_fad_picklist = geo_fad_picklist[columns]

In [22]:
geo_fad_picklist.columns

Index(['Name', '[Country (Admin Level 1)]', '[Country (Admin Level 2)]',
       '[Country (Admin Level 3)]', '[Country (Admin Level 4)]',
       '[Country (Admin Level 5)]', 'geometry'],
      dtype='object')

In [23]:
geo_gaz_merge = pd.merge(geo_fad_gaz, geo_fad_picklist, on = 'geometry', how = 'right') 

In [24]:
len(geo_fad_picklist)

29585

In [25]:
geo_gaz_merge.columns

Index(['FID', 'FID_', 'OBJECTID', 'Join_Count', 'TARGET_FID', 'Shape_Leng',
       'ADM4_EN', 'ADM4_UA', 'ADM4_RU', 'ADM4_PCODE', 'ADM3_EN', 'ADM3_UA',
       'ADM3_RU', 'ADM3_PCODE', 'ADM2_EN', 'ADM2_UA', 'ADM2_RU', 'ADM2_PCODE',
       'ADM1_EN', 'ADM1_UA', 'ADM1_RU', 'ADM1_PCODE', 'ADM0_EN', 'ADM0_UA',
       'ADM0_RU', 'ADM0_PCODE', 'lat', 'lon', 'osm_id', 'alt_name',
       'alt_name_e', 'place', 'population', 'postal_cod', 'Shape_Le_1',
       'Shape_Area', 'geometry', 'Name', '[Country (Admin Level 1)]',
       '[Country (Admin Level 2)]', '[Country (Admin Level 3)]',
       '[Country (Admin Level 4)]', '[Country (Admin Level 5)]'],
      dtype='object')

In [26]:
# Define filenames
filename_xlsx='UKR - Merge of Locations pick list values for FAD and Gazetteer of locations' + '.xlsx'

# Create a output path for the data
output_folder = r'UKR_data_sources/UKR_gazeteer/'

output_fp = os.path.join(output_folder, filename_xlsx)
geo_gaz_merge.to_excel(output_fp, encoding='utf8')